In [1]:
!pip install pyLDAvis --quiet
!pip install chart_studio --quiet


In [2]:
import pandas as pd
import numpy as np
import time
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from spacy.tokenizer import Tokenizer
import gensim.corpora as corpora
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
import plotly.express as px
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
from operator import itemgetter
from ipywidgets import interact
import tqdm
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets


# supress warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We will load in our preprocessed tweets from our [data cleaning notebook](https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Data_Cleaning_Feature_Engineering.ipynb). The csv can be found [here](https://drive.google.com/file/d/1-d-61YuocweY0F3rSnf-idL1ixbuKq4H/view?usp=sharing). 

In [4]:
# load in cleaned tweets from data cleaning notebook
tweets_cleaned = pd.read_json("/content/drive/MyDrive/Data/NLP_Capstone/motivational_tweets_cleaned.json")

In [5]:
tweets_cleaned.head()

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,2021-01-17 22:13:17,LewisHowes,Know this. Everything is happening for a reaso...,know happen reason favor betterment future pai...,"[know, happen, reason, favor, betterment, futu...",know_happen_reason favor betterment future pai...,"[know_happen_reason, favor, betterment, future...","[reason, favor, betterment, future, pain, feel...","[know, reason, favor, betterment, future, pain...",[know_happen_reason],know_happen_reason
61,2021-01-15 15:28:06,LewisHowes,Protect your inner peace at all costs. Create ...,protect inner peace cost create daily practice...,"[protect, inner, peace, cost, create, daily, p...",protect inner_peace cost create daily_practice...,"[protect, inner_peace, cost, create, daily_pra...","[peace, cost, practice, communicate, stress, d...","[peace, cost, practice, communicate, stress, d...","[inner_peace, daily_practice]",inner_peace daily_practice
161,2021-01-07 16:00:29,LewisHowes,Always remember to ask for exactly what you wa...,remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...",remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...","[health, abundance, peace, ask, wisdom, creati...","[remember, health, abundance, peace, ask, wisd...","[health_abundance, ask_wisdom]",health_abundance ask_wisdom
274,2021-01-01 02:44:46,LewisHowes,This will be your greatest year ever. All your...,great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...",great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...","[year, work, start, earn, love, embrace, fear,...","[year, work, start, pay, earn, love, embrace, ...",[massive_action],massive_action
317,2020-12-30 01:38:35,LewisHowes,Be grateful for the breakdown this year. It's ...,grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...",grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...","[year, breakthrough, money, mission, get, rela...","[year, set, breakthrough, money, come, mission...",[real_friend],real_friend


In [6]:
tweets_cleaned.applymap(type)

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
61,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
161,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
274,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
317,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...
977598,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977599,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977606,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977608,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>


In [7]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621449 entries, 44 to 977610
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      621449 non-null  datetime64[ns]
 1   user_name                 621449 non-null  object        
 2   content                   621449 non-null  object        
 3   content_preprocessed      621449 non-null  object        
 4   unigram_tokens            621449 non-null  object        
 5   ngrams                    621449 non-null  object        
 6   ngram_tokens              621449 non-null  object        
 7   nouns_only                621449 non-null  object        
 8   nouns_verbs               621449 non-null  object        
 9   bigrams_trigrams          621449 non-null  object        
 10  bigrams_trigrams_strings  621449 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 56.9+ MB


In [8]:
tweets_cleaned.shape

(621449, 11)

# Topic Modeling

In [9]:
# create dictionary
id2word = corpora.Dictionary(tweets_cleaned['nouns_verbs'])
# create texts corpus
texts = tweets_cleaned['nouns_verbs']
# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]
# print first 30 tuples from corpus
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [10]:
# number of topics
num_topics = 20
# build LDA model
base_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# # print the keyword in the 10 topics
# pprint(base_model.print_topics())
# doc_lda = base_model[corpus]

In [11]:
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
life way person matter david day thing care think mind

------ Topic 1 ------
life dyer power david soul create control action thinking thought

------ Topic 2 ------
dyer life way success amp thing opportunity matter happen future

------ Topic 3 ------
life heart churchill winston person problem rule happiness dream hell

------ Topic 4 ------
time life fall let person david rise feel john mind

------ Topic 5 ------
thing man david world person aurelius think time life power

------ Topic 6 ------
mind dyer goal step fact life amp david peace attract

------ Topic 7 ------
life thing word david change help stress happiness emerson look

------ Topic 8 ------
life look david time day way mirror change word dyer

------ Topic 9 ------
life love let aurelius time self truth speak man lao

------ Topic 10 ------
amp life love dyer act man find matter way change

------ Topic 11 ------
dream day life thing dyer result feel einstein way energy

------ Topic 12 ------

In [12]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=tweets_cleaned['nouns_verbs'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -6.716074277851314

Coherence Score:  0.4487116925229948


In [13]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.130197  0.197291       1        1  7.334916
13     0.030244 -0.010081       2        1  6.227934
17    -0.023816 -0.015887       3        1  6.171469
6     -0.128251 -0.047897       4        1  5.752833
2     -0.110226 -0.049275       5        1  5.689054
4      0.089218  0.143357       6        1  5.346047
11    -0.048439 -0.020498       7        1  5.327115
10    -0.090439 -0.054759       8        1  5.246205
1     -0.076532 -0.088537       9        1  5.183474
3     -0.032325  0.026745      10        1  5.016680
9     -0.025617  0.072257      11        1  4.817870
14     0.035006  0.002347      12        1  4.729776
0      0.023789  0.004967      13        1  4.719537
19     0.093784 -0.028135      14        1  4.626580
18     0.089177 -0.001162      15        1  4.574604
5      0.074161 -0.000068      16        1  4.391336
7     -0.000732 -0.053916      17        1  3.947657
12     0.056661 -0.004326      18        1  3.934183
8      0.082242 -0.067677      19        1  3.849865
15     0.092292 -0.004746      20        1  3.112864, topic_info=           Term          Freq         Total Category  logprob  loglift
318       david  58252.000000  58252.000000  Default  30.0000  30.0000
72        thing  45261.000000  45261.000000  Default  29.0000  29.0000
11         look  17968.000000  17968.000000  Default  28.0000  28.0000
1463       dyer  38390.000000  38390.000000  Default  27.0000  27.0000
97          try  12349.000000  12349.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
92           go    613.611686  12408.642358  Topic20  -4.9380   0.4628
11         look    658.261937  17968.544642  Topic20  -4.8678   0.1629
58       change    588.484384  21187.618635  Topic20  -4.9798  -0.1140
21    happiness    486.404080   8254.710130  Topic20  -5.1703   0.6381
98          day    500.154193  26950.246003  Topic20  -5.1425  -0.5172

[1496 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
5548      7  0.994304   abbad
1439      7  0.991279   abert
450       1  0.198397  accept
450       2  0.004157  accept
450       3  0.007558  accept
...     ...       ...     ...
700      19  0.014707     zig
5399      1  0.021808   zukav
5399     10  0.271611   zukav
5399     12  0.705791   zukav
2372      8  0.890860  zurich

[9284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 14, 18, 7, 3, 5, 12, 11, 2, 4, 10, 15, 1, 20, 19, 6, 8, 13, 9, 16])

In [14]:
get_document_topics = [base_model.get_document_topics(item) for item in corpus]

In [15]:
len(get_document_topics)

621449

In [16]:
get_document_topics[:20]

[[(4, 0.6131946), (12, 0.3338643)],
 [(2, 0.15983823), (7, 0.63149077), (16, 0.14328636)],
 [(10, 0.57430613), (11, 0.3564631)],
 [(0, 0.21321432), (2, 0.6215954), (16, 0.09980571)],
 [(6, 0.26057056), (10, 0.58053476), (16, 0.08806134)],
 [(6, 0.39537886), (10, 0.3546025), (13, 0.20001861)],
 [(5, 0.3410168), (10, 0.4726136), (19, 0.115536265)],
 [(3, 0.23232241), (10, 0.5876776)],
 [(5, 0.4670222), (7, 0.45115957)],
 [(5, 0.27518785), (10, 0.6348122)],
 [(9, 0.2283047), (10, 0.6966953)],
 [(7, 0.90500003)],
 [(16, 0.010000001), (17, 0.010000001), (18, 0.81000006)],
 [(0, 0.45705965), (12, 0.39294034)],
 [(0, 0.73108405), (6, 0.17891595)],
 [(4, 0.010000001), (10, 0.6069313), (19, 0.21306872)],
 [(0, 0.025),
  (1, 0.025),
  (2, 0.025),
  (3, 0.025),
  (4, 0.025),
  (5, 0.025),
  (6, 0.025),
  (7, 0.025),
  (8, 0.025),
  (9, 0.025),
  (10, 0.025),
  (11, 0.525),
  (12, 0.025),
  (13, 0.025),
  (14, 0.025),
  (15, 0.025),
  (16, 0.025),
  (17, 0.025),
  (18, 0.025),
  (19, 0.025)],
 [(2

In [17]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           alpha=a,
                                           eta=b,
                                           random_state=222)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [18]:
# start preprocess runtime
start_time = time.time() 

# Topics range
min_topics = 20
max_topics = 80
step_size = 5
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = [.01,.1,.5,1,'symmetric','asymmetric']
# Beta parameter
beta = [.01,.1,.5,1,'symmetric']

# model results dict
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=420)

    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
    lda_tuning_results = pd.DataFrame(model_results)                
    lda_tuning_results.to_csv('/content/drive/MyDrive/Data/NLP_Capstone/lda_tuning_20_plus_topics.csv', index=False)
    pbar.close()

# print preprocess runtime
print(time.strftime(f'%H hours, %M minutes, %S seconds', time.gmtime(time.time() - start_time)))

 86%|████████▌ | 360/420 [1:58:30<19:45, 19.75s/it]

01 hours, 58 minutes, 30 seconds


In [19]:
lda_tuning_results.sort_values('Coherence',ascending=False).head(30)

,Topics,Alpha,Beta,Coherence
269,60,asymmetric,symmetric,0.521515
299,65,asymmetric,symmetric,0.521234
266,60,asymmetric,0.1,0.520280
295,65,asymmetric,0.01,0.517416
146,40,asymmetric,0.1,0.516659
296,65,asymmetric,0.1,0.515638
145,40,asymmetric,0.01,0.515555
245,60,0.1,0.01,0.514674
149,40,asymmetric,symmetric,0.513614
335,75,0.1,0.01,0.513385


In [29]:
# number optimal of topics
num_topics = 60
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha='asymmetric',
                                        eta='symmetric',
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
step miracle world accept person time wilde word echo mind

------ Topic 1 ------
find leader spirit work dyer job vision c release life

------ Topic 2 ------
eye thing heart truth look conscience jonathan feel behavior find

------ Topic 3 ------
risk take tzu understanding order lao reputation change wisdom mark

------ Topic 4 ------
attract jim pain rohn covey stephen change circumstance peter learn

------ Topic 5 ------
stay direction happiness choice month wise speech life come memory

------ Topic 6 ------
proverb strike george time hepburn heart realise level sign mission

------ Topic 7 ------
life option thing inquire peter remain opinion amazing question amp

------ Topic 8 ------
create ralph emerson person dyer jump quit achievement self tracy

------ Topic 9 ------
search listen light succeed robert aspire ppl sun love amp

------ Topic 10 ------
dyer fall place thought presence life clement time stone result

------ Topic 11 ------
thing failure d

In [30]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['nouns_verbs'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -6.708353668373961

Coherence Score:  0.5161769647965639


In [31]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.134702  0.017878       1        1  3.421686
4      0.122110  0.024922       2        1  2.695192
1      0.138898  0.005468       3        1  2.523077
2      0.136370 -0.006581       4        1  2.460899
28    -0.045437 -0.037554       5        1  2.324835
15     0.026669  0.096329       6        1  2.323158
5      0.108861 -0.020622       7        1  2.252364
27     0.011090  0.008252       8        1  2.160917
13     0.042232  0.033963       9        1  2.158603
9      0.059717  0.023761      10        1  2.090822
36    -0.100635 -0.068390      11        1  2.089891
3      0.128890  0.016529      12        1  2.054954
49    -0.156725  0.123802      13        1  2.039626
56    -0.146016 -0.005920      14        1  2.005649
29    -0.000834 -0.069036      15        1  2.000221
31    -0.133319  0.123817      16        1  1.901301
14     0.079351 -0.034813      17        1  1.864475
8      0.129686  0.012089      18        1  1.855387
11     0.082810 -0.001524      19        1  1.844871
7      0.082906 -0.059274      20        1  1.844056
20     0.076399  0.014123      21        1  1.815763
12     0.056997  0.009284      22        1  1.812788
6      0.136184  0.026957      23        1  1.809669
39    -0.054478 -0.099778      24        1  1.773222
35    -0.055857  0.070337      25        1  1.682756
24     0.051353 -0.122235      26        1  1.672427
17     0.106407  0.078476      27        1  1.666065
46    -0.129392 -0.062131      28        1  1.648152
58    -0.093057  0.029363      29        1  1.648002
16     0.010815  0.014934      30        1  1.647081
37    -0.091365 -0.054744      31        1  1.619766
30    -0.022968  0.160123      32        1  1.595828
48    -0.057377  0.098580      33        1  1.551856
21     0.042952  0.063574      34        1  1.551668
47    -0.013542  0.044249      35        1  1.538512
38    -0.146794  0.035421      36        1  1.532340
51    -0.096779  0.017346      37        1  1.517970
23    -0.009478  0.125824      38        1  1.497485
42    -0.141312  0.077039      39        1  1.462234
25     0.060571 -0.082034      40        1  1.417382
54    -0.085349  0.028174      41        1  1.393660
18     0.084384 -0.004282      42        1  1.355230
32     0.004156 -0.056290      43        1  1.343682
19     0.113771  0.020028      44        1  1.327465
26     0.003228  0.019132      45        1  1.312606
40    -0.025192 -0.186697      46        1  1.285859
55    -0.145861  0.043303      47        1  1.283210
53    -0.126834 -0.064685      48        1  1.267853
33    -0.085075 -0.115608      49        1  1.250804
57    -0.143300 -0.146446      50        1  1.248047
22     0.036053  0.007559      51        1  1.240350
41    -0.054341 -0.072093      52        1  1.213409
34    -0.008927  0.001755      53        1  1.208865
50    -0.030043  0.155216      54        1  1.137107
10     0.144163 -0.002691      55        1  1.120144
43    -0.002578 -0.051987      56        1  1.093165
59     0.023719 -0.038285      57        1  1.020365
44    -0.033714  0.035843      58        1  0.993982
45    -0.004437 -0.093904      59        1  0.935100
52     0.005570 -0.105844      60        1  0.596146, topic_info=         Term          Freq         Total Category  logprob  loglift
42       love  20981.000000  20981.000000  Default  30.0000  30.0000
1463     dyer  43238.000000  43238.000000  Default  29.0000  29.0000
237   proverb   7892.000000   7892.000000  Default  28.0000  28.0000
32      dream  14506.000000  14506.000000  Default  27.0000  27.0000
28        ask   8389.000000   8389.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
58     change    223.965672  24282.840562  Topic60  -4.2931   0.4364
318     david    205.330344  31837.022205  Topic60  -4.3799   0.0787
28        ask    151.883439   8389.647912  Topic60

In [32]:
# number optimal of topics
num_topics = 40
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha='asymmetric',
                                        eta=.1,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
ask question heart work life courage search man bhajan nietzsche

------ Topic 1 ------
dream god discover life achievement heart character test reason responsibility

------ Topic 2 ------
treat winston man way tracy character judge answer friend life

------ Topic 3 ------
thing play man life albert know covey storm hand stephen

------ Topic 4 ------
life dare schuller mind attempt day w till blessing stevenson

------ Topic 5 ------
dyer grow risk self hold thing keep life worth wisdom

------ Topic 6 ------
peale vincent bind energy heart kennedy speak let confucius dwell

------ Topic 7 ------
time forget life man world mediocrity child decision freedom chain

------ Topic 8 ------
change happiness win step maya time attitude trouble design goethe

------ Topic 9 ------
fight consequence aim pull head person brian push deng xiaoping

------ Topic 10 ------
dyer place fall rule heart experience hell jefferson patience presence

------ Topic 11 ------
thing fa

In [33]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['nouns_verbs'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -6.733660263099588

Coherence Score:  0.5150786105875539


In [34]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.089817  0.037067       1        1  4.164312
1     -0.098754  0.041229       2        1  3.568566
13    -0.049906 -0.031162       3        1  3.415139
27     0.090245  0.106684       4        1  3.379079
5     -0.037902 -0.013366       5        1  3.369547
9      0.001177  0.017541       6        1  3.238755
2     -0.092022  0.044961       7        1  3.194333
4     -0.055452  0.043610       8        1  3.117313
3     -0.072771  0.058905       9        1  3.100937
28     0.159387  0.203213      10        1  2.883044
31     0.148775  0.054306      11        1  2.839460
23     0.126581 -0.084166      12        1  2.822133
7     -0.046534 -0.001256      13        1  2.796680
15     0.003223  0.037380      14        1  2.733396
6     -0.121158  0.043814      15        1  2.535166
34     0.109601 -0.077305      16        1  2.513765
11    -0.029417  0.015726      17        1  2.456982
16     0.063159  0.028828      18        1  2.388210
32    -0.023782  0.109528      19        1  2.367069
12     0.017754  0.039805      20        1  2.363030
21    -0.049602 -0.089910      21        1  2.340210
8     -0.084428  0.047074      22        1  2.324824
39     0.258012 -0.106916      23        1  2.307548
24    -0.065767  0.030480      24        1  2.303541
29    -0.019309  0.046908      25        1  2.223073
37     0.083876  0.006708      26        1  2.214906
10    -0.105779 -0.005500      27        1  2.203248
38     0.151051 -0.086772      28        1  2.197131
20    -0.051227  0.059131      29        1  2.187660
36     0.156540  0.148138      30        1  2.149414
26     0.051333 -0.041430      31        1  2.138841
33     0.079196 -0.160954      32        1  2.038811
14    -0.035139 -0.098444      33        1  2.006072
19    -0.020133  0.023301      34        1  1.997234
22    -0.044200 -0.035444      35        1  1.928704
17    -0.103652 -0.034288      36        1  1.913271
18    -0.046727 -0.002094      37        1  1.782261
35    -0.023913 -0.093690      38        1  1.767960
30    -0.003215 -0.153230      39        1  1.411870
25    -0.129307 -0.128410      40        1  1.316507, topic_info=         Term          Freq         Total Category  logprob  loglift
42       love  20043.000000  20043.000000  Default  30.0000  30.0000
98        day  26572.000000  26572.000000  Default  29.0000  29.0000
324  strength   8355.000000   8355.000000  Default  28.0000  28.0000
237   proverb   6928.000000   6928.000000  Default  27.0000  27.0000
318     david  39112.000000  39112.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
159     world    613.985440  21199.661691  Topic40  -4.0769   0.7884
577      word    386.349403  12556.545945  Topic40  -4.5401   0.8489
114       way    389.465194  24786.966557  Topic40  -4.5320   0.1769
70      heart    324.026640  16416.200028  Topic40  -4.7160   0.4050
348  question    285.245667   6013.759004  Topic40  -4.8435   1.2817

[2316 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
5571     15  0.019064  aabye
5571     19  0.705376  aabye
5571     35  0.247835  aabye
2939      1  0.173009  aaron
2939      2  0.037073  aaron
...     ...       ...    ...
5399     18  0.069882  zukav
5399     23  0.216520  zukav
5399     28  0.701114  zukav
5399     31  0.001146  zukav
5399     37  0.001146  zukav

[19775 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 14, 28, 6, 10, 3, 5, 4, 29, 32, 24, 8, 16, 7, 35, 12, 17, 33, 13, 22, 9, 40, 25, 30, 38, 11, 39, 21, 37, 27, 34, 15, 20, 23, 18, 19, 36, 31, 26])